In [1]:
import sys
sys.path.append("../../")

import torch
import torchvision.utils
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import optim
from torch.utils.data import DataLoader
from torchvision import models
from torchsummary import summary

import numpy as np
import cv2
import pandas as pd
from sklearn.metrics import accuracy_score

import src.neuralNetworksArch.BstCnnFull as bst
import src.neuralNetworksArch.Mpkp as mpkp

import src.dataPreparation.CreatePartial as create_partial
import src.utils.Visual as vis
import src.utils.Checkpoint as ckp
import src.utils.Metrics as metrics

from src.config.Param import *

In [2]:
MODEL_PATH = '../../models/FULL BST CUHK02.pth'
PATH = '../../dataset/testing/same_cam/'
DATATEST_PATH = PATH + 'testing.csv'
IMAGES_PATH = PATH + '/images/full/'
IMAGES_OCCL_PATH = PATH + '/images/occl_20/'

In [9]:
def cv_image2tensor(img, transform, convert=True):
    img = cv2.resize(img, (64, 128))
    if convert:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = transform(img)
    img = [img]
    img_tensors = torch.stack(img)

    return img_tensors

In [10]:
model = {}
model_arch  = bst.BstCnnFull()
checkpoint = ckp.load_checkpoint(load_dir=MODEL_PATH)
model_arch.load_state_dict(checkpoint['state_dict'])
model_arch.eval()

model['model'] = model_arch
model['max_dist'] = checkpoint['max_dist']
model['threshold'] = checkpoint['threshold']

In [11]:
trans = transforms.Compose([transforms.ToTensor()])
df = pd.read_csv(DATATEST_PATH)
y_true = []
y_pred = []

verbose = False
with torch.no_grad():
    for index, data in df.iterrows():
        if verbose == True:
            print('No-{}'.format(index+1))
            print('-'*50)
        img1 = cv2.imread(IMAGES_PATH + data['image_1'])
        img2 = cv2.imread(IMAGES_PATH + data['image_2'])
        label = data['label']
        
        input1 = cv_image2tensor(img1, trans)
        input2 = cv_image2tensor(img2, trans)
        out1, out2 = model['model'](input1, input2)
        euclidean_distance = F.pairwise_distance(out1, out2)

        dist = metrics.normalize_dist(euclidean_distance.item(), model['max_dist'])
        pred = 0.0 if dist <= model['threshold'] else 1.0
        y_pred.append(pred)
        y_true.append(float(label))
        
        if verbose == True:
            print('actual distance => {}'.format(dist))
            print('actual thresh => {}'.format(model['threshold']))
            print('actual label => {}'.format(label))
        
        if verbose == True:
            concatenated = torch.cat((cv_image2tensor(img1, trans), cv_image2tensor(img2, trans)),0)
            vis.imshow(torchvision.utils.make_grid(concatenated))
            print('='*50, end='\n\n')

acc = accuracy_score(np.array(y_true), np.array(y_pred))
print('Accuracy : {}'.format(acc))

Accuracy : 1.0
